# Representing Covalent Labelling Information

One of the ideas we had was a method to represent residues from something like covalent labelling approaches for idenitfying surface residues. This gives a simple implementation for that.

This uses the example of Creatine Kinase (which I had covalent labelling data for), PDB:2CRK.

In [54]:
class RepLabels:
    """
    A class to output iCn3D commands for surface labelling information
    """
    def __init__(self,pdb_code,pdb,labels):
        self.pdb_code = pdb_code
        self.labels = self.get_labels(labels)
        self.atom_coords = self.get_ids(pdb)
        
    def get_labels(self,labels):
        """Parse covalent labels and return residue ids (currently ignoring chains)"""
        mls = set()
        
        for line in (open(labels)).read().splitlines():
            mls.add(int(line.split('|')[0]))
            
        return mls
    
    def get_ids(self,pdb):
        """As I couldn't figure out what command to use to select a single residue,
        this method finds CA atom identifier for each residue."""
        ca_coords = {}

        with open(pdb) as f:
            g = f.read().splitlines()
            for line in [i for i in g if i.startswith("ATOM") and i[13:16].strip()=="CA"]:
                aa = int(line[22:28].strip())
                atom = int(line[4:13])
                ca_coords[aa] = atom
            f.close()
            
        return ca_coords
    
    def output_statefile(self,state_pref="./",state_suff="_state.txt",color="0FF"):
        atom_sel = []

        atom_sel.append("""load mmdb %s""" % self.pdb_code)

        for ml in mls:
            atom_sel.append("""pickatom %d""" % self.atom_coords[ml])
            atom_sel.append("""style sidec ball and stick""")
            atom_sel.append("""color %s""" % color)
        
        with open("%s%s%s" % (state_pref,self.pdb_code,state_suff),"w+") as f:
            for line in atom_sel:
                f.write(f"{line}\n")
            f.close()

In [55]:
rep_lab = RepLabels("2crk","./covalent_labels/2crk.pdb","./covalent_labels/ml.txt")
rep_lab.output_statefile(state_pref="./covalent_labels/")

When the resulting statefile is loaded into icn3d it looks like this:

![Sreeenshot](covalent_labels/screenshot.png)